# Analytic Equation of State Library

## Detailed Analysis

This script is used to provide a comprehensive library of analytic Equation of State (EoS) models for the hadronic phase of neutron stars. It serves as the central repository for the physics functions that define how energy density varies with pressure ($\epsilon(P)$) for various nuclear theory parameterizations (e.g., SLy, APR, MDI).

Additionally, the script defines a unified multi-layer crust model based on the Douchin & Haensel (SLy) parameterization. These functions are mathematically differentiable, allowing the exact computation of the speed of sound without numerical finite-difference errors.

## Physics and Math

The Equation of State connects the pressure $P$ to the energy density $\epsilon$. This relationship is required to close the system of Tolman-Oppenheimer-Volkoff (TOV) differential equations.

### Thermodynamic Relations
The code defines $\epsilon$ as an analytic function of $P$. To solve the tidal deformability equations, the speed of sound squared ($c_s^2$) is also required. This is related to the derivative of the energy density:

$$
c_s^2 = \frac{dP}{d\epsilon} = \left( \frac{d\epsilon}{dP} \right)^{-1}
$$

The script uses symbolic mathematics to compute $\frac{d\epsilon}{dP}$ analytically.

### Core Parameterizations
The hadronic core models are based on piecewise analytic fits to numerical data from nuclear many-body theory (often matching the form found in Read et al. 2009). A typical fit function used in the code resembles:

$$
\epsilon(P) = A \cdot P^\alpha + B \cdot P^\beta + \dots
$$

or exponential saturation forms for certain models:

$$
\epsilon(P) = C + D \left(1 - e^{-P/\lambda}\right) + \dots
$$

where $A, B, C, D, \lambda, \alpha, \beta$ are fitting constants specific to each nuclear model (e.g., "MDI-1", "APR-1").

### Crust Model
The crust is modeled as four distinct layers, handling the transition from the low-density envelope to the core. The functional forms include log-polynomials for the outer envelope and exponential fits for the inner crust.

## Code Walkthrough

### 1. Core Model Definitions
A dictionary named `core_exprs` is defined using `sympy` symbols. Each entry represents a distinct nuclear physics model.

```python
p = Symbol('p')

# Analytic fits for Energy Density epsilon(p) [MeV/fm^3]
core_exprs = {
    "MDI-1": 4.1844 * p**0.81449 + 95.00135 * p**0.31736,
    # ... other models ...
    "APR-1": 0.000719964 * p**1.85898 + 108.975 * p**0.340074,
}
```
These expressions map pressure ($p$) to energy density ($\epsilon$). The constants are derived from literature fits.

### 2. Multi-Layer Crust Definition
The crust is divided into four regions (`crust1` through `crust4`). These functions allow the EoS to remain realistic at low densities, where matter consists of nuclei rather than uniform nuclear fluid.

```python
# Crust 4: Envelope (Log-Polynomial)
c = [31.93753, 10.82611, 1.29312, 0.08014, 0.00242, 0.000028]
logP = sym_log(p, 10) 
crust4 = 10**(c[0] + c[1]*logP + c[2]*logP**2 + ...)
```
The exact pressure thresholds that separate these crust layers are not defined here; they are handled in `src.const.CONSTANTS` and applied in the TOV solver logic.

### 3. Lambdification (Symbolic to Numeric)
The symbolic expressions are converted into fast NumPy functions. Crucially, the derivative is computed symbolically (`e.diff(p)`) before conversion.

```python
# Returns tuple (eps_val, dedp_val)
core_funcs = {
    k: (lambdify(p, e, 'numpy'), lambdify(p, e.diff(p), 'numpy')) 
    for k, e in core_exprs.items()
}
```
This loop creates a dictionary where each key maps to a tuple:
1.  The function for energy density $\epsilon(P)$.
2.  The function for the derivative $\frac{d\epsilon}{dP}(P)$.

This ensures that `dedp` is calculated exactly, avoiding the instability of numerical differentiation during the ODE integration steps.